Imports

In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np

TF-IDF class. Contains methods which can easily access relevant information from the csv file which the class reads, which represents the TF-IDF model.

TF-IDF stands for Term Frequency - Inverse Document Frequency. It gives each keyword-document combination a weight, based on how often it occurs in the document, but also based on how often it occurs in all documents. This ensures that words which occur often in many documents do not receive high weights, rather only words which are relatively unique to a document and occur frequently in that document get high weights.

If all documents are about topic A, we do not care that document x contains is about document A.

Implementation wise, the TF-IDF is a csv file with each row representing a document and each column representing a keyword, the value at i,j represents the weight that keyword j has in document i.

In [2]:
class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("../data/tf_idf/tf_idf.csv")

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("../data/tf_idf/tf_idf_keywords.csv")
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

Instantiate the object

In [3]:
tfidf = TfIdf()

Read in the dataset of visits, used to test the information need prediction.

In [4]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('../data/clickdata/dataNoUnscrapedVisitsOrUnder20Sec.csv', converters=conv)

Create seperate dataframes for the paths taken and for the seconds spent on each url.
In paths, index 0 gives a list of urls, the equivalent index 0 in seconds represents how many seconds the user spent on the equal index urls.

In [15]:
paths = df.url_id_path
print(len(paths))

3583310


Simplified implementation of IUNIS, which simply looks at the keywords which the user comes across, does not use the adjacency matrix, proximal cues, and spreading activation.
It seems that these are mostly important for WUFIS, which we are not interested in, thus implementing it is not worth the effort.

Input is the ordered list of page id's that the user has visited.
For each page, take the TF-IDF values from that page, which is a row of keywords, with either 0 if the keyword is not present or [0,1] if it is.

Then, for each next page, we add the TF-IDF values onto the existing weights.

Eventually we can sort the list and we will have the keywords with the highest weight on top, which then represents the most relevant keywords in the visit up until now, and acts as our prediction for what the user is looking for.
Instead of only looking at the user alone, we can also compare their information need to information needs of other users, and by looking at where the other users end up, we can predict where our user might want to end up.

Optionally, a decay factor can be introdced. Essentially, every iteration, all values are divided by a factor, meaning that keywords on recent pages are more biased and weighted more heavily.

It is also possible to give more weight to pages based on how much time the user spent on that page.

In [9]:
from pandas import DataFrame
#Method which essentially just takes the keywords and their weights on each page the user visits, continues to the next url, and then sums the weights.
#Be sure to realise that only passing a path will execute the normal implementation, which simply sums all new tf-idf values of each url.
#This is because the default decay_factor value is 1, meaning that nothing changes, and the default secs is None, if that is None, the if statement with secs is never entered and we do not take time into account.
#Sorted gives an option to sort the vector before returning, this puts the most relevant keywords on top, but does not allow for easy similarity comparison, thus should only be used if this is not the intention
def find_keyword_weights(path, decay_factor=1, secs=None, sorted=False):
    #initiate array of weights (weights start as 0, size=num of keywords)
    weights = np.zeros(tfidf.get_number_of_keywords())
    #ensures all output is visible
    pd.options.display.max_rows = 0
    #Iterate over each page url
    for i in range(len(path)):
        #Divide the weights by the decay factor (1 by default, no decay in that case)
        weights = weights/decay_factor
        #Initialize the tf-idf weights of the current page
        new_weights = tfidf.get_all_keywords_by_id_normalized(path[i])
        #If we are taking time into account, multiply the weights by the amount of time spent on this page
        if secs is not None:
            new_weights*=secs[i]
        #Add the new weights to the existing weights
        weights += new_weights
        #Normalize such that the largest value becomes 1
        weights /= max(weights)
    #Option to sort s.t. max weights are on top.
    if sorted:
        df = DataFrame(weights, columns=['weights'])
        return df.sort_values(by=['weights'], ascending=False)
    return weights

Method which calculates keywords of a visit

In [13]:
import time

def compute_keywords_of_complete_visit(min_visit_size, paths):
    start = time.perf_counter()
    count=0
    for path in paths:
        if(count==1000):
            print(time.perf_counter()-start)
        print(count)
        path[:] = [x for x in path if x<=1555]
        if len(path)>min_visit_size:
            keywords = find_keyword_weights(path, decay_factor=7)
            #This is the keyword vector of a visit, with the currently most optimal settings.
        count+=1

In [14]:

compute_keywords_of_complete_visit(5, paths)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


C:\Users\01din\AppData\Local\Temp\ipykernel_16268\1285695263.py:32: RuntimeWarning: invalid value encountered in true_divide
  keywords_weights_normalized = keywords_weights/sum_value


122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371


KeyboardInterrupt: 